In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
sys.path.insert(0, '../')
sys.path.insert(0,"../../scripts/")

import numpy as np

import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200
# import textract

from tabula import read_pdf
import camelot

%reload_ext autoreload
%autoreload 2



# from scripts import manipulate

In [303]:
dd = read_pdf('../../files/ldo/ldo_2021.pdf', pages='121-268' ,multiple_tables=True, encoding='utf-8', lattice=False)

In [326]:
df = pd.DataFrame()

for i in range(len(dd)):
    if len(dd[i].columns)>2:
        print(i)
        dd[i] = dd[i][dd[i].columns[:2]]
    dd[i].columns = [0,1]
    df = pd.concat([df,dd[i]], axis=0)
    
df = df.reset_index(drop=True)

mask = (df[1].fillna('').str.contains(r'\b\d', regex=True)) & (~df[1].fillna('').str.contains(r'META', regex=True))
df[1] = np.where(mask,df[1],np.nan)
mask = (df[1].fillna('').str.contains(r'\b\d\d\d\d', regex=True))
df[0] = np.where(mask,df[0] +'\n' +df[1] ,df[0])
df[1] = np.where(mask,np.nan,df[1])

In [327]:
# def re_filter(value):  
#     return re.findall(_filter, value)
def generate_col(df,_filter):
    mask = df[0].fillna('').str.contains(_filter,regex=True, case=True)
    return df[mask][0]

def fill_col(df, _filter):
    mask = df['root'].fillna('').str.contains(_filter,regex=True, case=True)
    return np.where(mask, df[0],np.nan)

In [331]:
df["programa"] = generate_col(df, r"\bPrograma:\n")
df["orgao"] = generate_col(df, r"\bÓrgão:\n")
df["indicador_programa"] = generate_col(df, r"\bINDICADORES DE RESULTADO DE PROGRAMA")
df["produto"] = generate_col(df, r"\bPRODUTO: ")
df["indicador_produto"] = generate_col(df, r"\bINDICADOR DE PRODUTO")
df["acao"] = generate_col(df, r"\bAÇÃO\b")

df["root"] = (
    df["programa"].fillna("")
    + df["orgao"].fillna("")
    + df["indicador_programa"].fillna("")
    + df["produto"].fillna("")
    + df["indicador_produto"].fillna("")
    + df["acao"].fillna("")
)
df["root"] = np.where(df["root"] == "", np.nan, df["root"])
df["root"] = df["root"].fillna(method="ffill")



df["programa"] = fill_col(df, r"\bPrograma:\n")
df["orgao"] = fill_col(df, r"\bÓrgão:\n")
df["indicador_programa"] = fill_col(df, r"\bINDICADORES DE RESULTADO DE PROGRAMA")
df["produto"] = fill_col(df, r"\bPRODUTO: ")
df["indicador_produto"] = fill_col(df, r"\bINDICADOR DE PRODUTO")
df["acao"] = fill_col(df, r"\bAÇÃO\b")
df["acao"] = df[
    df["acao"].fillna("").str.contains(r"\b\d\d\d\d -", regex=True, case=True)
]["acao"]




finalidade_progama_mask = (
    df["programa"].fillna("").str.contains(r"\bPrograma:\n", regex=True, case=True)
)
df["finialidade_programa"] = df[~finalidade_progama_mask]["programa"]
df["programa"] = np.where(finalidade_progama_mask, df["programa"], np.nan)

finalidade_produto_mask = (
    df["produto"].fillna("").str.contains(r"\bPRODUTO: ", regex=True, case=True)
)
df["finialidade_produto"] = df[~finalidade_produto_mask]["produto"]
df["produto"] = np.where(finalidade_produto_mask, df["produto"], np.nan)


for col in df.columns[2:]:
    if col != "acao":
        df[col] = df[col].fillna(method="ffill")
    else:
        df[col] = df[col].fillna(method="bfill")


value_programa_mask = (
    df["root"]
    .fillna("")
    .str.contains(r"\bINDICADORES DE RESULTADO DE PROGRAMA", regex=True, case=True)
)
df["value_indicador_programa"] = np.where(value_programa_mask, df[1], np.nan)
df["value_indicador_programa"] = (
    df["value_indicador_programa"].str.replace(".", "").str.replace(",", ".")
)
df["value_indicador_programa"] = pd.to_numeric(df["value_indicador_programa"], errors="coerce")


In [332]:


value_produto_mask = (
    df["root"].fillna("").str.contains(r"\bINDICADOR DE PRODUTO", regex=True, case=True)
)
df["value_indicador_produto"] = np.where(value_produto_mask, df[1], np.nan)
df["value_indicador_produto"] = (
    df["value_indicador_produto"].str.replace(".", "").str.replace(",", ".")
)
df["value_indicador_produto"] = pd.to_numeric(df["value_indicador_produto"], errors="coerce",downcast='float')


df_programa = df[df["value_indicador_programa"].notnull()]
cols = [
    "programa",
    "orgao",
    "indicador_programa",
    "finialidade_programa",
    "value_indicador_programa",
]
df_programa = df_programa[cols]
df_programa['programa'] = df_programa['programa'].str.replace('Programa:\n','')
df_programa['orgao'] = df_programa['orgao'].str.replace('Órgão:\n','')
for col in df_programa.columns[:-1]:
    df_programa[col] = df_programa[col].str.upper().str.replace('\r','')

    
    
cols = [
    "programa",
    "orgao",
    "finialidade_programa",
    "produto",
    "finialidade_produto",
    "indicador_produto",
    "acao",
    "value_indicador_produto",
]
df_produto = df[df["value_indicador_produto"].notnull()]
df_produto = df_produto[cols]

df_produto['programa'] = df_produto['programa'].str.replace('Programa:\n','')
df_produto['orgao'] = df_produto['orgao'].str.replace('Órgão:\n','')
for col in df_produto.columns[:-1]:
    df_produto[col] = df_produto[col].str.upper().str.replace('\r','')

In [333]:
df_programa.to_csv('../../data/ldo/ldo_2021_programa.csv', index=False)
df_produto.to_csv('../../data/ldo/ldo_2021_produto.csv', index=False)

In [301]:
df_programa[df_programa['programa']=='1218 - FOMENTO CULTURAL']

,programa,orgao,indicador_programa,finialidade_programa,value_indicador_programa
1398,1218 - FOMENTO CULTURAL,12000 - SECRETARIA DA CULTURA E ECONOMIA CRIATIVA,NÚMERO DE EDITAIS LANÇADOS PELO PROGRAMA (UNID...,FINALÍSTICO,33.0
1399,1218 - FOMENTO CULTURAL,12000 - SECRETARIA DA CULTURA E ECONOMIA CRIATIVA,NÚMERO DE MUNICÍPIOS CONTEMPLADOS NO PROGRAMA ...,FINALÍSTICO,180.0
1400,1218 - FOMENTO CULTURAL,12000 - SECRETARIA DA CULTURA E ECONOMIA CRIATIVA,VALOR APORTADO NO PROGRAMA DE FOMENTO CULTURAL...,FINALÍSTICO,112360000.0


## LDO 2022

In [121]:
dd = read_pdf('../../files/ldo/ldo_2022.pdf', 
              pages='151-174' ,
              multiple_tables=True, 
              encoding='utf-8', 
              lattice=True
             )

In [122]:
df = pd.DataFrame()

for i in range(len(dd)):
    if len(dd[i].columns)>6:
        print(i)
        dd[i] = dd[i][dd[i].columns[:6]]
    dd[i].columns = [0,1,2,3,4,5]
    df = pd.concat([df,dd[i]], axis=0)
    
df = df.reset_index(drop=True)

mask = (df[0].fillna('').str.contains(r'\bPrograma:', regex=True))
df[0] = np.where(mask,df[0]+df[1],df[0])
# mask = (df[1].fillna('').str.contains(r'\b\d\d\d\d', regex=True))
# df[0] = np.where(mask,df[0] +'\n' +df[1] ,df[0])
# df[1] = np.where(mask,np.nan,df[1])

In [123]:
# def re_filter(value):  
#     return re.findall(_filter, value)
def generate_col(df,_filter,col):
    mask = df[col].fillna('').str.contains(_filter,regex=True, case=True)
    return df[mask][col]

def fill_col(df, _filter, col):
    mask = df['root'].fillna('').str.contains(_filter,regex=True, case=True)
    return np.where(mask, df[col],np.nan)

In [124]:
df['programa'] = generate_col(df,r'\bPrograma:',0)
df['orgao'] = generate_col(df,r'\bÓrgão:',0)
df['objetivo_estrategico'] = generate_col(df,r'\bOBJETIVOS ESTRATÉGICOS ASSOCIADOS',0)
df['diagnostico'] = generate_col(df,r'\bDIAGNÓSTICO PROGRAMA:',0)
df['objetivo'] = generate_col(df,r'\bOBJETIVO DO PROGRAMA',0)
df['publico_alvo'] = generate_col(df,r'\bPÚBLICO ALVO:',0)
df['abrangencia_espacial'] = generate_col(df,r'\bABRANGÊNCIA ESPACIAL:',0)
df['indicador_programa'] = generate_col(df,r'INDICADORES DE RESULTADO DE PROGRAMA:',0)
df['produto'] = generate_col(df,r'PRODUTO:',0)
df['indicador_produto'] = generate_col(df,r'\bINDICADOR DE PRODUTO',0)


df["root"] = (
    df["programa"].fillna("")
    + df["orgao"].fillna("")
    + df["objetivo_estrategico"].fillna("")
    + df["diagnostico"].fillna("")
    + df['objetivo'].fillna("")
    + df["publico_alvo"].fillna("")
    + df["abrangencia_espacial"].fillna("")
    + df["indicador_programa"].fillna("")
    + df["produto"].fillna("")
    + df["indicador_produto"].fillna("")
)
df["root"] = np.where(df["root"] == "", np.nan, df["root"])
df["root"] = df["root"].fillna(method="ffill")


df['programa'] = fill_col(df,r'\bPrograma:',0)
df['orgao'] = fill_col(df,r'\bÓrgão:',1)
mask_orgao = df['orgao'].fillna('').str.contains(r'\d\b -',regex=True, case=True)
df['orgao'] = np.where(mask_orgao, df['orgao'],np.nan)




df['objetivo_estrategico'] = fill_col(df,r'\bOBJETIVOS ESTRATÉGICOS ASSOCIADOS',1)
df['diagnostico'] = fill_col(df,r'\bDIAGNÓSTICO PROGRAMA:',1)
df['objetivo'] = fill_col(df,r'\bOBJETIVO DO PROGRAMA',1)
df['publico_alvo'] = fill_col(df,r'\bPÚBLICO ALVO:',1)
df['abrangencia_espacial'] = fill_col(df,r'\bABRANGÊNCIA ESPACIAL:',1)
df['indicador_programa'] = fill_col(df,r'INDICADORES DE RESULTADO DE PROGRAMA:',0)
df['produto'] = fill_col(df,r'PRODUTO:',0)
df['indicador_produto'] = fill_col(df,r'\bINDICADOR DE PRODUTO',0)


finalidade_progama_mask = (
    df["programa"].fillna("").str.contains(r"\bPrograma:", regex=True, case=True)
)
df["finialidade_programa"] = df[~finalidade_progama_mask]["programa"]
df["programa"] = np.where(finalidade_progama_mask, df["programa"], np.nan)

finalidade_produto_mask = (
    df["produto"].fillna("").str.contains(r"\bPRODUTO: ", regex=True, case=True)
)
df["finialidade_produto"] = df[~finalidade_produto_mask]["produto"]
df["produto"] = np.where(finalidade_produto_mask, df["produto"], np.nan)




for col in df.columns[6:]:
    df[col] = df[col].fillna(method="ffill")

df['valor_mais_recente_programa'] = fill_col(df,r'INDICADORES DE RESULTADO DE PROGRAMA:',1)
df['periodo_transferencia_programa'] = fill_col(df,r'INDICADORES DE RESULTADO DE PROGRAMA:',2)
df['fonte_informacao_programa'] = fill_col(df,r'INDICADORES DE RESULTADO DE PROGRAMA:',3)
df['meta_final_ppa_programa'] = fill_col(df,r'INDICADORES DE RESULTADO DE PROGRAMA:',4)

df['valor_mais_recente_produto'] = fill_col(df,r'\bINDICADOR DE PRODUTO',1)
df['periodo_transferencia_produto'] = fill_col(df,r'\bINDICADOR DE PRODUTO',2)
df['fonte_informacao_produto'] = fill_col(df,r'\bINDICADOR DE PRODUTO',3)
df['meta_final_ppa_produto'] = fill_col(df,r'\bINDICADOR DE PRODUTO',4)

cols = [
    'valor_mais_recente_programa',
    'meta_final_ppa_programa',
    'valor_mais_recente_produto',
    'meta_final_ppa_produto'
]

for col in cols:
    df[col] = df[col].str.replace(".", "").str.replace(",", ".")
    df[col] = pd.to_numeric(df[col], errors="coerce")


In [125]:
df_programa = df[df['meta_final_ppa_programa'].notnull()]
cols = [
    'programa',
    'orgao',
    'objetivo_estrategico',
    'diagnostico',
    'objetivo',
    'publico_alvo',
    'abrangencia_espacial',
    'finialidade_programa',
    'indicador_programa',
    'fonte_informacao_programa',
    'periodo_transferencia_programa',
    'valor_mais_recente_programa',
    'meta_final_ppa_programa'
]
for col in cols[:-3]:
    df_programa[col] = df_programa[col].str.replace('\r','').str.replace('Programa:','').str.upper()
    
df_programa = df_programa[cols]

In [126]:
df_produto = df[df['meta_final_ppa_produto'].notnull()]

cols = [
    'programa',
    'orgao',
    'publico_alvo',
    'abrangencia_espacial',
    'finialidade_programa',
    'produto',
    'finialidade_produto',
    'indicador_produto',
    'fonte_informacao_produto',
    'periodo_transferencia_produto',
    'valor_mais_recente_produto',
    'meta_final_ppa_produto',
]
for col in cols[:-3]:
    df_produto[col] = df_produto[col].str.replace('\r','').str.replace('Programa:','').str.upper()
    
df_produto = df_produto[cols]

In [128]:
df_programa.to_csv('../../data/ldo/ldo_2022_programa.csv', index=False)
df_produto.to_csv('../../data/ldo/ldo_2022_produto.csv', index=False)

In [129]:
df_produto

,programa,orgao,publico_alvo,abrangencia_espacial,finialidade_programa,produto,finialidade_produto,indicador_produto,fonte_informacao_produto,periodo_transferencia_produto,valor_mais_recente_produto,meta_final_ppa_produto
37,0815 - GESTÃO INSTITUCIONAL DA SECRETARIA DA EDUCAÇÃO,08000 - SECRETARIA DA EDUCAÇÃO,ALUNOS E PROFISSIONAIS DA REDE ESTADUAL DE EDUCAÇÃO.,ESTADO,FINALÍSTICO,PRODUTO: ALIMENTAÇÃO ESCOLAR OFERECIDA AOS ALUNOS DA REDE ESTADUAL,FINALÍSTICO,NÚMERO DE ALUNOS ATENDIDOS PELO SERVIÇO DE ALIMENTAÇÃOESCOLAR (UNIDADE),INEP - CENSO ESCOLAR/SEE-SAESP 2/SED (SECRETARIAESCOLAR DIGITAL),2016-2018,2377053.00,2.400000e+06
42,0815 - GESTÃO INSTITUCIONAL DA SECRETARIA DA EDUCAÇÃO,08000 - SECRETARIA DA EDUCAÇÃO,ALUNOS E PROFISSIONAIS DA REDE ESTADUAL DE EDUCAÇÃO.,ESTADO,FINALÍSTICO,PRODUTO: TRANSPORTE ESCOLAR REGULAR E ESPECIALIZADO GARANTIDO,FINALÍSTICO,PERCENTUAL DE ALUNOS DA REDE ESTADUAL TRANSPORTADOS (%),"""SEDUC /CISE/ DEPART. DESERVIÇOS DE TRANSPORTEE ASSISTÊNCIA AO ALUNO /CENTRO DE TRANSPORTE ES",2018,11.00,1.100000e+01
47,0815 - GESTÃO INSTITUCIONAL DA SECRETARIA DA EDUCAÇÃO,08000 - SECRETARIA DA EDUCAÇÃO,ALUNOS E PROFISSIONAIS DA REDE ESTADUAL DE EDUCAÇÃO.,ESTADO,FINALÍSTICO,PRODUTO: REDE FÍSICA ESCOLAR ADEQUADA PARA A APRENDIZAGEM A PARTIR DO CURRÍCULO PAULISTA,FINALÍSTICO,NÚMERO DE INTERVENÇÕES DE INFRAESTRUTURA FÍSICA NOS PRÉDIOSDA REDE ESTADUAL DE ENSINO (UNIDADE),SEDUC/ CISE,2016-2018,2901.00,3.825000e+03
48,0815 - GESTÃO INSTITUCIONAL DA SECRETARIA DA EDUCAÇÃO,08000 - SECRETARIA DA EDUCAÇÃO,ALUNOS E PROFISSIONAIS DA REDE ESTADUAL DE EDUCAÇÃO.,ESTADO,FINALÍSTICO,PRODUTO: REDE FÍSICA ESCOLAR ADEQUADA PARA A APRENDIZAGEM A PARTIR DO CURRÍCULO PAULISTA,FINALÍSTICO,PERCENTUAL DE SERVIÇOS DE MANUTENÇÃO ATENDIDOS NO TOTALDOS SERVIÇOS SOLICITADOS NO ANO (%),SEDUC-CISE,2016-2018,100.00,1.000000e+02
53,0815 - GESTÃO INSTITUCIONAL DA SECRETARIA DA EDUCAÇÃO,08000 - SECRETARIA DA EDUCAÇÃO,ALUNOS E PROFISSIONAIS DA REDE ESTADUAL DE EDUCAÇÃO.,ESTADO,FINALÍSTICO,"PRODUTO: EQUIPAMENTOS, MOBILIÁRIO,SUPRIMENTO,SERVIÇOS E KIT ESCOLAR PARA O NOVO CURRÍCULO",FINALÍSTICO,PERCENTUAL DE ESCOLAS ATENDIDAS COM REPOSIÇÃO DE MATERIALPERMANENTE E PARA CONSUMO PARA APOIO À APRENDIZAGEM (%),SEDUC-DIRETORIAS DEENSINO/ GEMAT,NaN,NaN,1.000000e+02
65,0815 - GESTÃO INSTITUCIONAL DA SECRETARIA DA EDUCAÇÃO,08000 - SECRETARIA DA EDUCAÇÃO,ALUNOS E PROFISSIONAIS DA REDE ESTADUAL DE EDUCAÇÃO.,ESTADO,FINALÍSTICO,PRODUTO: RECURSOS DE TECNOLOGIA DE INFORMAÇÃO E COMUNICAÇÃO PARA PROJETOS PRIORITÁRIOS,FINALÍSTICO,PERCENTUAL DE UNIDADES DA REDE ESTADUAL BENEFICIADAS PELAAQUISIÇÃO DE EQUIPAMENTOS E RECURSOS DE TECNOLOGIA DACOMUNICAÇÃO (%),SEDUC/CITEM,2018,53.00,8.500000e+01
70,0815 - GESTÃO INSTITUCIONAL DA SECRETARIA DA EDUCAÇÃO,08000 - SECRETARIA DA EDUCAÇÃO,ALUNOS E PROFISSIONAIS DA REDE ESTADUAL DE EDUCAÇÃO.,ESTADO,FINALÍSTICO,PRODUTO: PROFISSIONALIZAÇÃO E GESTÃO DE PESSOAS COM FOCO NA APRENDIZAGEM,MELHORIA DE GESTÃO,PERCENTUAL DE ABSENTEÍSMO DOS PROFISSIONAIS DA EDUCAÇÃO (%),SEDUC/ RELATÓRIO BFE/HOLERITE,NaN,NaN,1.107000e+01
71,0815 - GESTÃO INSTITUCIONAL DA SECRETARIA DA EDUCAÇÃO,08000 - SECRETARIA DA EDUCAÇÃO,ALUNOS E PROFISSIONAIS DA REDE ESTADUAL DE EDUCAÇÃO.,ESTADO,FINALÍSTICO,PRODUTO: PROFISSIONALIZAÇÃO E GESTÃO DE PESSOAS COM FOCO NA APRENDIZAGEM,MELHORIA DE GESTÃO,PERCENTUAL DE PROFESSORES QUE CUMPREM TODA A SUA JORNADAEM UMA MESMA ESCOLA ESTADUAL (%),SISTEMA DE CADASTROCARGA HORÁRIA,2018,24.70,3.400000e+01
76,0815 - GESTÃO INSTITUCIONAL DA SECRETARIA DA EDUCAÇÃO,08000 - SECRETARIA DA EDUCAÇÃO,ALUNOS E PROFISSIONAIS DA REDE ESTADUAL DE EDUCAÇÃO.,ESTADO,FINALÍSTICO,PRODUTO: PARCERIAS ESTADO-MUNICÍPIOS PARA CONSTRUÇÕES ESCOLARES,MELHORIA DE GESTÃO,NÚMERO DE CRECHES CONSTRUÍDAS (UNIDADE),SEDUC - CISE,2016-2018,243.00,1.110000e+03
77,0815 - GESTÃO INSTITUCIONAL DA SECRETARIA DA EDUCAÇÃO,08000 - SECRETARIA DA EDUCAÇÃO,ALUNOS E PROFISSIONAIS DA REDE ESTADUAL DE EDUCAÇÃO.,ESTADO,FINALÍSTICO,PRODUTO: PARCERIAS ESTADO-MUNICÍPIOS PARA CONSTRUÇÕES ESCOLARES,M